In [1]:
!pip install python-dotenv


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
import pandas as  pd
import requests
import numpy as np
from flipside import Flipside
import os
from dotenv import load_dotenv


In [3]:
# Config
columns = ['unique_trader', 'platform', 'blockchain','trading_vol', 'vol_all_platform', 'interactions_count', 'last_interact_date', 'last_date_all_platform', 'activity']

## Perpetual Trades

In [24]:
load_dotenv()

query_id = {
    "vertex": 3319571,
    "hmx": 3322162,
    "mux":3331840,
    "lvl":3329599,
    "synthetix":3331775,
    "gmx":3326970,
    "dydx":3350149,
    "aevo":3332234,
    "gns":3329779,
    'hyperliquid':3351533
}
dune_api = os.getenv('DUNE_API_2')
df_all = pd.DataFrame()
for platform,id in query_id.items():
    url = f"https://api.dune.com/api/v1/query/{str(id)}/results?api_key={dune_api}"
    response = requests.get(url)
    df = pd.DataFrame(response.json())
    data = df.iloc[-1,-1]
    df= pd.DataFrame(data)
    df= df[columns]
    # download
    df.to_excel(f"dataset/traders_{platform}.xlsx", index= False)
    print(f"donwload of {platform} data")
    # concatenate
    df_all = pd.concat([df_all,df], axis=0)
    

df_all.to_excel("dataset/concatenate.xlsx", index = False)
df_all

donwload of vertex data
donwload of hmx data
donwload of mux data
donwload of lvl data
donwload of synthetix data
donwload of gmx data
donwload of dydx data
donwload of aevo data
donwload of gns data
donwload of hyperliquid data


,unique_trader,platform,blockchain,trading_vol,vol_all_platform,interactions_count,last_interact_date,last_date_all_platform,activity
0,0xaf5e2af64e5926875fa5f3d5e6ac272fc602224e,Vertex,arbitrum,1.461988e+10,None,156191,2024-03-01,None,active
1,0x5494c260fd71799414d174aa802750efa2dca5dd,Vertex,arbitrum,1.327859e+10,None,149346,2024-03-01,None,active
2,0x351d1d0ed37f6854bfc36ba24caa529688990f6e,Vertex,arbitrum,4.595251e+09,None,157022,2024-03-01,None,active
3,0x0c173177a769e2b9a8aed05105da6a34918c6bf5,Vertex,arbitrum,2.522704e+09,None,22650,2024-03-01,None,active
4,0xc3239fbac3ecaff2165f5a27089fcf5d94c37fed,Vertex,arbitrum,1.458955e+09,None,51891,2024-03-01,None,active
...,...,...,...,...,...,...,...,...,...
1423,0x5f44fe1c8b5d0802eda2a9b638c6163ad52d633b,Hyperliquid,optimism,1.000021e+04,None,3,2023-12-01,None,active
1424,0xcfe122de90515685f806e79b29ff9351ce764098,Hyperliquid,optimism,1.000016e+04,None,1,2023-12-05,None,active
1425,0x9866f56b785d38e67eefed374821d1d7ae9ef18a,Hyperliquid,optimism,1.000013e+04,None,1,2023-10-13,None,active
1426,0x0681e5bee2f420bc5a1754f09e0dedc0c5b4be85,Hyperliquid,optimism,1.000006e+04,None,2,2023-12-05,None,active


In [6]:
# Final Layout

pivot_df = df_all.pivot(index='unique_trader', columns='platform', values='trading_vol').fillna(0)
pivot_df['Cumulative Trading Volume'] = df_all.groupby('unique_trader')['trading_vol'].sum()
pivot_df['Count of Platforms Used'] = df_all.groupby('unique_trader')['platform'].nunique()
pivot_df['Number of Interactions'] = df_all.groupby('unique_trader')['interactions_count'].sum()
pivot_df['Date of Last Interaction'] = df_all.groupby('unique_trader')['last_interact_date'].max()
pivot_df.reset_index(inplace=True)


# Renaming the trading volume cols
platform_columns = [col for col in pivot_df.columns if col not in ['unique_trader', 'Cumulative Trading Volume', 'Count of Platforms Used', 'Number of Interactions', 'Date of Last Interaction']]
for col in platform_columns:
    pivot_df.rename(columns={col: f'Trading Volume {col}'}, inplace=True)

#pivot_df.to_excel("dataset/perpetual_dex.xlsx", index= False)  # or to_excel
pivot_df


platform,unique_trader,Trading Volume AAEVO,Trading Volume DYDX,Trading Volume GMX Trade,Trading Volume GNS,Trading Volume HMX,Trading Volume Hyperliquid,Trading Volume LVL Finance,Trading Volume MUX,Trading Volume Synthetix,Trading Volume Vertex,Cumulative Trading Volume,Count of Platforms Used,Number of Interactions,Date of Last Interaction
0,0x00000000000057667bfefe3158822e6fb4058a6d,0.0,0.0,58084.768813,0.0000,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,5.808477e+04,1,3,2023-09-29
1,0x00000000000e8716b4e2d12400a01474cdfad760,0.0,0.0,0.000000,0.0000,0.0,0.0,7.527903e+06,0.0,0.0,0.000000e+00,7.527903e+06,1,1900,2023-09-27
2,0x000000000036277944858dfe6b9ae18c929e570c,0.0,0.0,0.000000,619294.1472,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,6.192941e+05,1,10,2024-01-09
3,0x00000000007cee88e01241c9db0a02b859db5b52,0.0,0.0,366570.200819,0.0000,0.0,0.0,4.952040e+05,0.0,0.0,0.000000e+00,8.617742e+05,2,1685,2024-03-01
4,0x00000000051cbce3fd04148cce2c0adc7c651829,0.0,0.0,0.000000,0.0000,0.0,0.0,0.000000e+00,0.0,0.0,2.552827e+04,2.552827e+04,1,7,2024-02-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49647,0xfffb43b6b6048adc32ded4fcedc429b17e7ea72d,0.0,0.0,29969.889880,0.0000,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,2.996989e+04,1,15,2023-10-02
49648,0xfffb737987e5358243c76b88354f9a0462662481,0.0,0.0,0.000000,115988.2000,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,1.159882e+05,1,2,2024-02-14
49649,0xfffb92fb89ae51c421018e42a5ca6e7336314438,0.0,0.0,0.000000,0.0000,0.0,0.0,0.000000e+00,0.0,0.0,1.024285e+06,1.024285e+06,1,68,2023-11-16
49650,0xfffc49959ae5d909d810c0a135260e08e1d9d678,0.0,0.0,0.000000,33396.0000,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,3.339600e+04,1,5,2024-02-08


In [10]:
data

[]

In [8]:
# Enrich perpetual traders table
# retrieve trader adresses vs type(contract or wallet)
url = "https://flipsidecrypto.xyz/api/v1/queries/1f6630a4-7d0c-4f70-a218-d603f5a094e9/data/latest"
response = requests.get(url)
data = response.json()
df_contract= pd.DataFrame(data)
display(df_contract)

df_perp = pd.merge(pivot_df, df_contract, how = 'left', left_on = 'unique_trader', right_on = 'UNIQUE_TRADER')


""


KeyError: 'UNIQUE_TRADER'

In [11]:
df_perp = df_perp.drop(columns = ['UNIQUE_TRADER'])
df_perp.to_excel("dataset/perpetual_dex.xlsx", index= False)
df_perp

NameError: name 'df_perp' is not defined

## NFT Community

In [12]:
#Extract data prepared from Flipside
url = "https://flipsidecrypto.xyz/api/v1/queries/e3cced65-01d9-48b9-b164-71a2675e3990/data/latest"
response = requests.get(url)
data = response.json()
df_nft= pd.DataFrame(data)
df_nft

,WALLET,NFT_NAME,COUNT,WALLET_BALANCE_USD
0,0xb7f7f6c52f2e2fdb1963eab30438024864c313f6,CRYPTOPUNKS,822,0.99
1,0x29469395eaf6f95920e59f858042f0e28d98a20b,PudgyPenguins,421,17879.01
2,0xa858ddc0445d8131dac4d1de01f834ffcba52ef1,CRYPTOPUNKS,419,168695.11
3,0x29469395eaf6f95920e59f858042f0e28d98a20b,Kanpai Pandas,317,17879.01
4,0xcb2a8334143176e6eb4863e7ab3ed5f3258ee94b,Merit Circle - Tactile NFT,240,119200.17
...,...,...,...,...
14766,0xbb2ef97280d90690eee44c7dba2a1b8c61ff10e1,PudgyPenguins,1,16098.03
14767,0x9a15687cb70c44f30f9c79f5edc48b4e2e47e4ee,CRYPTOPUNKS,1,166.68
14768,0x55ce859d074ea5316d0d16f5695ed7d2f0f2c1f8,Kanpai Pandas,1,57.12
14769,0xf246046f6199cf4dea18f3a5235c0b355b56e925,Kanpai Pandas,1,3993.13


## Whales and OGs

###  TOP LProviders  on Uniswap & Curve AN (Ethereem and Arbitrum)

In [13]:
url = "https://flipsidecrypto.xyz/api/v1/queries/28a95d3b-2c12-42e7-97bf-aa2ac4c30e41/data/latest"
response = requests.get(url)
data = response.json()
df_dex= pd.DataFrame(data)
df_dex

,WALLET,PLATFORM,ACTIVITY,AMOUNT_USD,IS_ETH_CONTRACT,IS_ARB_CONTRACT
0,0x00000000ff0027b3f3d2b0412d4a4ced4e3366da,Uniswap & Curve,active,7.081190e+06,Yes,No
1,0x000013de30d1b1d830dcb7d54660f4778d2d4af5,Uniswap & Curve,active,4.386262e+09,No,No
2,0x000bae21577c39cbb1e4efca262b6775ec1ac4c9,Uniswap & Curve,active,6.166160e+06,No,No
3,0x000000002fcecf3bc87acc58802cdb526c4f019d,Uniswap & Curve,active,3.018134e+06,No,No
4,0x0000a441fbb1fbaadf246539bf253a42abd31494,Uniswap & Curve,active,2.278823e+07,No,No
5,0x0000b8e312942521fb3bf278d2ef2458b0d3f243,Uniswap & Curve,active,2.227337e+09,No,No
6,0x0034daf2e65f6ef82bc6f893dbbfd7c232a0e59c,Uniswap & Curve,active,2.895872e+06,No,No
7,0x002525a77d262dd11ec63c9146a821b11564ce29,Uniswap & Curve,active,2.760395e+06,No,No
8,0x0035fc5208ef989c28d47e552e92b0c507d2b318,Uniswap & Curve,active,9.382001e+06,No,No
9,0x00000000002088951336d7972746a135f2956417,Uniswap & Curve,active,3.937381e+06,No,No


### Staking Protocol such as Lido & Rocket Pool

In [14]:
url = "https://flipsidecrypto.xyz/api/v1/queries/b3239ae5-92c8-495d-99cc-9ac17b155dc8/data/latest"
response = requests.get(url)
data = response.json()
df_stake= pd.DataFrame(data)
df_stake

,WALLET,PLATFORM,ACTIVITY,AMOUNT_USD,IS_ETH_CONTRACT
0,0x6decd32f5a2ab9b43bc6ad8923eb6d2395de145b,Lido & Rocketpool,active,6.572020e+06,Yes
1,0x7d2b63a9ab475397d9c247468803f25cf6523b76,Lido & Rocketpool,active,7.976273e+06,Yes
2,0x5f6ae08b8aeb7078cf2f96afb089d7c9f51da47d,Lido & Rocketpool,active,1.093010e+09,Yes
3,0xe4614d81bd262d62b09867c915d556c508b7036f,Lido & Rocketpool,active,2.654377e+06,Yes
4,0xc7be520a13dc023a1b34c03f4abdab8a43653f7b,Lido & Rocketpool,active,9.941589e+06,Yes
...,...,...,...,...,...
2126,0xe2b3d79592c1397b7bffbfd00ea6029ccba0cdad,Lido & Rocketpool,active,1.100376e+06,No
2127,0xb5a02e4a4e33e28a06713278e1edc451eb3e784f,Lido & Rocketpool,active,1.236952e+06,No
2128,0xe4eef4c9ee425a0cbfeb49146183694230438fcd,Lido & Rocketpool,active,3.586030e+06,No
2129,0x4292e45ca23a9d2792504bfba374ba38fe2643c5,Lido & Rocketpool,active,2.632915e+06,No


### MAkerDAO gov

In [15]:
url = "https://flipsidecrypto.xyz/api/v1/queries/8a8bd36e-e0a5-4170-a3c5-7fd16a50dbe6/data/latest"
response = requests.get(url)
data = response.json()
df_maker= pd.DataFrame(data)
df_maker

,WALLET,PLATFORM,ACTIVITY,AMOUNT_USD
0,0xe21125f4c35f11a1b4978c9cf9a7de4d2ecea406,Optimism Gov.,active,NaN
1,0x0f6f713711be3251326e70d6ec1d35793beadad2,Optimism Gov.,active,NaN
2,0x24f356a9c4ddf1f003d786b7c2740c10583be53f,Optimism Gov.,active,NaN
3,0x475efac48a0a18660a7a26ee6bd5febf466930f8,Optimism Gov.,active,NaN
4,0xfbec297af3f4925966ab295a0771c420a9e85e08,Optimism Gov.,active,NaN
...,...,...,...,...
271,0xf00132a6cbf2c5a1f515fca2f41f9d5f945519cc,Optimism Gov.,active,10.66
272,0xe45fb071103bdf01d991b49c8f065d451e193679,Optimism Gov.,active,3.94
273,0x4a4bb53985ce40539b5cabdc6a9ae8fdf72d0a73,Optimism Gov.,active,3.94
274,0x5dc44a74b90b6221415f41f4fcfd652512139a03,Optimism Gov.,active,3.94


### veCRV Holders

In [16]:
url = "https://flipsidecrypto.xyz/api/v1/queries/ba79099c-7652-4730-8e73-c077fd3e28bf/data/latest"
response = requests.get(url)
data = response.json()
df_crv= pd.DataFrame(data)
df_crv

,WALLET,PLATFORM,ACTIVITY,AMOUNT_USD,IS_ETH_CONTRACT
0,0x8fa7cd7daf391af3d9e816196dfa8e064d9208ec,Curve Gov.,active,1051.476511,No
1,0x120a265fbb08c07579264f825a536bec683dcd8c,Curve Gov.,active,2278.199720,No
2,0x70c3c1d59ac5791e1c7d1b5613c8574ba077c081,Curve Gov.,active,1015.741968,No
3,0xb390593bbb06f6ef4b1b4a5a42757b847bda26ae,Curve Gov.,active,1365.598480,No
4,0xafab1adfb25e919d9f6ee7196f1ad0ffa93cad12,Curve Gov.,active,1617.342109,No
...,...,...,...,...,...
2848,0x4f6fff5cd87eaf8cf09467366fe783e7ecb1317b,Curve Gov.,active,1578.319700,No
2849,0x163c1d864e91900f1993f57f5eafee36f14b9cd2,Curve Gov.,active,2054.687038,No
2850,0x115d0d8822efc9607c52b04628098b48970b993b,Curve Gov.,active,8324.250075,No
2851,0x8ea42952c97f8d9af8f242d7640925a9e18d752d,Curve Gov.,active,3033.158649,No


### Optimism Governance

In [19]:
url = "https://flipsidecrypto.xyz/api/v1/queries/4c6616d8-9808-40fa-9321-ccbe80bb20b9/data/latest"
response = requests.get(url)
data = response.json()
df_opt= pd.DataFrame(data)
df_opt

""


### Governance (DEX/Stake/Chains)


In [20]:
url = "https://flipsidecrypto.xyz/api/v1/queries/631b19a0-46ac-4f2a-8a36-37be4cc41df2/data/latest"
response = requests.get(url)
data = response.json()
df_gov= pd.DataFrame(data)
df_gov['IS_ETH_CONTRACT'] = np.nan
df_gov

,WALLET,PLATFORM,ACTIVITY,AMOUNT_USD,IS_ETH_CONTRACT
0,0xc71a6d83214a0ca8d5d9a111bb4c6a2e754e0083,Aave Gov.,active,NaN,NaN
1,0x37258b36862a93286b1a2a503e73ece0943bc87b,Aave Gov.,active,NaN,NaN
2,0x7ecbe6ace4424d9e6d38126faea63851e8ad70f1,Aave Gov.,active,NaN,NaN
3,0xc830e2b3be2881d935d1bc11cc2440adec0b65cf,Aave Gov.,active,NaN,NaN
4,0x067f24592eaa4fc6264d4a788921d75c05d5856d,Aave Gov.,active,NaN,NaN
...,...,...,...,...,...
59260,0x97b7a9aa3179c5a1148165ab2af541bcad030d38,GMX Gov.,active,0.02,NaN
59261,0xaeb8d54c846610e7fde3b75b49c9a9e84b45da4d,GMX Gov.,active,0.02,NaN
59262,0x90b0190a6d95e79829c31419567739dd06f80c2d,GMX Gov.,active,0.02,NaN
59263,0x7c96e329cc7951fdd9c532918de2e87481a63371,GMX Gov.,active,0.01,NaN


### Gitcoin Grant Donors

In [21]:
dune_api = os.getenv("DUNE_API")
url = f"https://api.dune.com/api/v1/query/3351720/results?api_key={dune_api}"
response = requests.get(url)
df = pd.DataFrame(response.json())
data = df.iloc[-1,-1]
df_gitcoin= pd.DataFrame(data)
df_gitcoin = df_gitcoin[['wallet', 'platform', 'activity', 'amount_usd', 'is_eth_contract']]
df_gitcoin.columns = df_gitcoin.columns.str.upper()
df_gitcoin

,WALLET,PLATFORM,ACTIVITY,AMOUNT_USD,IS_ETH_CONTRACT
0,0x63dc8d03505f4c3e44d08447ea67f8ab6ddbde9e,Gitcoin Grants,active,0.029432,None
1,0x91815bb64aeb18418b9475f2f4716615164295b8,Gitcoin Grants,active,1.092169,None
2,0x977fca0b3be31ba6866f89f71217b3692a68d892,Gitcoin Grants,active,0.000000,None
3,0x52966b8c65b67f98aaf702133b13993df333943d,Gitcoin Grants,active,0.090330,None
4,0xd81aebd5454e15a723954290ac9c2970397ce1fe,Gitcoin Grants,active,0.067525,None
...,...,...,...,...,...
270,0x127f814029584d9de09cb261548120925a09b290,Gitcoin Grants,active,0.000000,None
271,0x6ab048f2b6d5cde86da0001d7bb206063f3d2446,Gitcoin Grants,active,1.118952,None
272,0xce588b80f435f8dbef9a545088c5175beba0f235,Gitcoin Grants,active,0.000000,None
273,0xec383fe0e0b50f6d8675d31cb75b8c875894a4b0,Gitcoin Grants,active,0.042789,None


## Final Layouts

In [22]:
df_og = pd.concat([df_stake, df_dex, df_maker, df_crv, df_opt, df_gov, df_gitcoin], axis = 0)
df_og.to_excel('dataset/og_whale_users.xlsx', index = False)
df_og

,WALLET,PLATFORM,ACTIVITY,AMOUNT_USD,IS_ETH_CONTRACT,IS_ARB_CONTRACT
0,0x6decd32f5a2ab9b43bc6ad8923eb6d2395de145b,Lido & Rocketpool,active,6.572020e+06,Yes,NaN
1,0x7d2b63a9ab475397d9c247468803f25cf6523b76,Lido & Rocketpool,active,7.976273e+06,Yes,NaN
2,0x5f6ae08b8aeb7078cf2f96afb089d7c9f51da47d,Lido & Rocketpool,active,1.093010e+09,Yes,NaN
3,0xe4614d81bd262d62b09867c915d556c508b7036f,Lido & Rocketpool,active,2.654377e+06,Yes,NaN
4,0xc7be520a13dc023a1b34c03f4abdab8a43653f7b,Lido & Rocketpool,active,9.941589e+06,Yes,NaN
...,...,...,...,...,...,...
270,0x127f814029584d9de09cb261548120925a09b290,Gitcoin Grants,active,0.000000e+00,None,NaN
271,0x6ab048f2b6d5cde86da0001d7bb206063f3d2446,Gitcoin Grants,active,1.118952e+00,None,NaN
272,0xce588b80f435f8dbef9a545088c5175beba0f235,Gitcoin Grants,active,0.000000e+00,None,NaN
273,0xec383fe0e0b50f6d8675d31cb75b8c875894a4b0,Gitcoin Grants,active,4.278896e-02,None,NaN


In [23]:
# NFT community final layout
perp_traders =  list(set(pivot_df.unique_trader.values))
og_whale_list =  list(set(df_og.WALLET.values))

df_nft['is_perp_trader'] = df_nft.WALLET.apply(lambda x: 'Yes' if x in perp_traders else 'No')
df_nft['is_og_whale'] = df_nft.WALLET.apply(lambda x: 'Yes' if x in og_whale_list else 'No')
df_nft.to_excel('dataset/nft_community.xlsx', index = False)
df_nft

,WALLET,NFT_NAME,COUNT,WALLET_BALANCE_USD,is_perp_trader,is_og_whale
0,0xb7f7f6c52f2e2fdb1963eab30438024864c313f6,CRYPTOPUNKS,822,0.99,No,No
1,0x29469395eaf6f95920e59f858042f0e28d98a20b,PudgyPenguins,421,17879.01,No,No
2,0xa858ddc0445d8131dac4d1de01f834ffcba52ef1,CRYPTOPUNKS,419,168695.11,No,No
3,0x29469395eaf6f95920e59f858042f0e28d98a20b,Kanpai Pandas,317,17879.01,No,No
4,0xcb2a8334143176e6eb4863e7ab3ed5f3258ee94b,Merit Circle - Tactile NFT,240,119200.17,No,No
...,...,...,...,...,...,...
14766,0xbb2ef97280d90690eee44c7dba2a1b8c61ff10e1,PudgyPenguins,1,16098.03,No,No
14767,0x9a15687cb70c44f30f9c79f5edc48b4e2e47e4ee,CRYPTOPUNKS,1,166.68,No,No
14768,0x55ce859d074ea5316d0d16f5695ed7d2f0f2c1f8,Kanpai Pandas,1,57.12,No,No
14769,0xf246046f6199cf4dea18f3a5235c0b355b56e925,Kanpai Pandas,1,3993.13,No,No


## Summary

In [123]:
df_con = pd.read_excel('dataset/concatenate.xlsx')
df_con

,unique_trader,platform,blockchain,trading_vol,vol_all_platform,interactions_count,last_interact_date,last_date_all_platform,activity
0,0xaf5e2af64e5926875fa5f3d5e6ac272fc602224e,Vertex,arbitrum,1.135845e+10,NaN,112298,2024-01-06,NaN,active
1,0x5494c260fd71799414d174aa802750efa2dca5dd,Vertex,arbitrum,9.769726e+09,NaN,105567,2024-01-06,NaN,active
2,0x351d1d0ed37f6854bfc36ba24caa529688990f6e,Vertex,arbitrum,2.724428e+09,NaN,110840,2024-01-06,NaN,active
3,0xd604cfa918a7fcf24fd5789483323cdcb37b7fcf,Vertex,arbitrum,4.762204e+08,NaN,24047,2024-01-06,NaN,active
4,0x519e19eac9ead404343c68224dd2a22c5c25f992,Vertex,arbitrum,4.017094e+08,NaN,4454,2024-01-03,NaN,active
...,...,...,...,...,...,...,...,...,...
49918,0xcfe122de90515685f806e79b29ff9351ce764098,Hyperliquid,optimism,1.000016e+04,NaN,1,2023-12-05,NaN,active
49919,0x9866f56b785d38e67eefed374821d1d7ae9ef18a,Hyperliquid,optimism,1.000013e+04,NaN,1,2023-10-13,NaN,active
49920,0x0681e5bee2f420bc5a1754f09e0dedc0c5b4be85,Hyperliquid,optimism,1.000006e+04,NaN,2,2023-12-05,NaN,active
49921,0xc18120b4b22c78ea55f8a7acbd7d7082d3c73a2b,Hyperliquid,optimism,1.000002e+04,NaN,2,2023-12-08,NaN,active


In [141]:
summary_perpetual = df_con.groupby(by = 'platform').agg({'unique_trader': 'count'})
display(summary_perpetual )
summary_nft = df_nft.groupby(by = 'NFT_NAME').agg({'WALLET': 'count'})
display(summary_nft )
summary_og = df_og.groupby(by = 'PLATFORM').agg({'WALLET': 'count'})
display(summary_og)

,unique_trader
platform,
AAEVO,1063
DYDX,2355
GMX Trade,22940
GNS,5356
HMX,1279
Hyperliquid,1533
LVL Finance,330
MUX,5009
Synthetix,4586


,WALLET
NFT_NAME,
CRYPTOPUNKS,3433
CryptoDickbutts S3,1819
Cryptotester,873
Kanpai Pandas,2107
Merit Circle - Tactile NFT,150
PudgyPenguins,4187
lobsterdao,1879


,WALLET
PLATFORM,
Aave Gov.,32680
Curve Gov.,2782
ENS Gov,1307
GMX Gov.,17514
Gitcoin Gov,5529
Gitcoin Grants,533
Lido & Rocketpool,1931
MakerDAO Gov.,316
Optimism,20786
